In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [32]:
# import OSNIR dataset
df = pd.read_csv('../Data/OSNIR_values_extendedv3_new datasetbcsv.csv')
# shuffling rows of OSNIR dataframe and reset indexes
df = df.sample(frac=1)
df.reset_index(inplace=True, drop=True)

In [33]:
df.head()

,OSNIRnumerical(dB),Ns,Pch(dBm),L(km),B(GHz),GB(GHz),Nch
0,4.280959,30,-10,100,25.0,0.0,9
1,18.186455,5,-3,100,25.0,0.0,9
2,8.661015,43,3,100,50.0,0.0,9
3,24.221112,8,-4,50,25.0,0.0,9
4,22.722819,11,-5,50,25.0,0.0,3


1800 values for training (0,1800-1)\
600 values for validation (1800,2400-1)\
600 values for testing (2400,3000-1)

In [34]:
data = np.array(df)

In [35]:
# input x aka Ns, Pch, L, B, GB, Nch values
x_train = data[0:1800-1, 1:6]
x_valid = data[1800:2400-1, 1:6]
x_test = data[2400:3000-1, 1:6]

# output y aka OSNIR values
y_train = data[0:1800-1, 0]
y_valid = data[1800:2400-1, 0]
y_test = data[2400:3000-1, 0]

### ---- WIP ----

In [76]:
model = keras.Sequential([
    keras.layers.Dense(32, input_shape=(5,), activation='relu'),  # input layer (1)
    keras.layers.Dense(32, activation='relu'),  # hidden layer (2)
    keras.layers.Dense(1, activation='relu') # output layer (3)
])

In [92]:
# define y true and pred
def osnr_loss(y_true,y_pred):
    squared_difference = tf.square(y_train - y_valid)
    return tf.reduce_mean(squared_difference, axis=-1)

In [93]:
model.compile(optimizer='adam',
              loss='osnr_loss',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=50)

In [ ]:
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))